In [1]:
#this is how we are going to fetch the files, this is how the workflow is going to be, uploaded file will be sent to our bucket.
#and the GS util URI will be returned which can be sent to the Vertex AI API for processing.

from google.cloud import storage

def upload_blob_and_get_uri(bucket_name, source_file_name, destination_blob_name, project_id):
    """Uploads a file to the specified bucket and returns its gs:// URI."""
    # Pass the project_id explicitly
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    gs_uri = f"gs://{bucket_name}/{destination_blob_name}"

    print(f"File {source_file_name} uploaded to {destination_blob_name} in bucket {bucket_name}.")
    print(f"GS URI: {gs_uri}")

    return gs_uri


In [2]:
your_bucket_name = "legal-doc-bucket1"  # Replace with your actual bucket name
local_file_path = r"C:\\Users\\user\\OneDrive\\Documents\\GenAI-exchange\\backend\\pdf\\generative-ai_pdf_certificate_of_incoporation.pdf" # Replace with the path to the file you want to upload
destination_object_name = "generative-ai_pdf_certificate_of_incoporation.pdf" # The name you want the file to have in the bucket
project_id = "sodium-coil-470706-f4" # Use your actual project ID

# Call the function to upload and get the URI
uploaded_uri = upload_blob_and_get_uri(
    your_bucket_name,
    local_file_path,
    destination_object_name,
    project_id
)

if uploaded_uri:
    print(f"\nSuccessfully obtained GS URI: {uploaded_uri}")


c:\Users\user\OneDrive\Documents\GenAI-exchange\backend\venv\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


File C:\\Users\\user\\OneDrive\\Documents\\GenAI-exchange\\backend\\pdf\\generative-ai_pdf_certificate_of_incoporation.pdf uploaded to generative-ai_pdf_certificate_of_incoporation.pdf in bucket legal-doc-bucket1.
GS URI: gs://legal-doc-bucket1/generative-ai_pdf_certificate_of_incoporation.pdf

Successfully obtained GS URI: gs://legal-doc-bucket1/generative-ai_pdf_certificate_of_incoporation.pdf


In [3]:
from google import genai
from google.genai import types
import base64

def generate():
  client = genai.Client(
      vertexai=True,
      project="sodium-coil-470706-f4",
      location="global", 
  )

  document1 = types.Part.from_uri(
      file_uri=uploaded_uri,  # already available
      mime_type="application/pdf",
  )
  
  # ⬇️ your most recent "user message"
  text1 = types.Part.from_text(text="""What's the exact date that the certificate of incorporation / authority be revoked? Explain your logic.""")

  # ⬇️ the system instructions, as-is
  si_text1 = """\"You are a highly reliable legal document assistant specialized in answering questions about legal documents.\",
      \"Your role is to extract, summarize, and clarify content from provided documents without altering meaning.\",
      \"Always ground your responses in the text of the document. If the document does not contain the answer, state clearly that the information is not available.\",
      \"Do not provide legal advice or personal opinions. Remain neutral and factual.\",
      \"When referencing information, cite the relevant section, clause, or paragraph from the document whenever possible.\",
      \"Use plain and concise language, but preserve legal precision.\",
      \"If a question is ambiguous, ask for clarification rather than assuming.\""""

  model = "gemini-2.5-flash-lite"

  # ⬇️ Instead of conversation history, just pass the most recent message
  contents = [
    types.Content(
      role="user",
      parts=[document1, text1]
    )
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 65535,
    safety_settings = [
      types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
      types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
      types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
      types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
    ],
    system_instruction=[types.Part.from_text(text=si_text1)],
    thinking_config=types.ThinkingConfig(thinking_budget=0),
  )

  # collect response manually in one variable
  response_text = ""
  for chunk in client.models.generate_content_stream(
    model=model,
    contents=contents,
    config=generate_content_config,
  ):
    if chunk.text:
      response_text += chunk.text
      print(chunk.text, end="")

  # ⬇️ now you have "just the recent message + its response" manually
  print("\n\n--- Manual Log ---")
  print("User:", text1.text)
  print("Bot:", response_text)

generate()

The document states that the Certificate of Incorporation/Authority will be revoked "after 60 days from the date of this notice" (Section: R.I.G.L. 7-1.2-1310 and 7-1.2-1414). The date of the notice is June 19, 2023. Therefore, the certificate would be revoked after July 19, 2023.

--- Manual Log ---
User: What's the exact date that the certificate of incorporation / authority be revoked? Explain your logic.
Bot: The document states that the Certificate of Incorporation/Authority will be revoked "after 60 days from the date of this notice" (Section: R.I.G.L. 7-1.2-1310 and 7-1.2-1414). The date of the notice is June 19, 2023. Therefore, the certificate would be revoked after July 19, 2023.


In [3]:
import base64
from google import genai
from google.genai import types
# import utils # You would need to provide the content of utils.py here

# Placeholder for utils.py content. You must replace this with the actual functions from your utils.py
# For demonstration, I'll include a simple placeholder for what these functions might look like.
# In a real scenario, you'd paste your utils.py content directly or import it.
class utils:
    @staticmethod
    def validate_key(request):
        # Your actual validation logic from utils.py
        # For now, let's assume no validation for this pure Python version
        return None

    @staticmethod
    def get_parts_from_message(message_content):
        # Your actual logic to convert message content to parts from utils.py
        # Example: if message_content is a string, return [types.Part.from_text(text=message_content)]
        if isinstance(message_content, str):
            return [types.Part.from_text(text=message_content)]
        # Handle other types if your original utils.py does.
        return []

    @staticmethod
    def convert_content_to_gr_type(content):
        # Your actual logic to convert genai content to Gradio compatible type from utils.py
        # Since we're removing Gradio, this function will now just return the text content
        # or a representation suitable for your Flask app.
        # For simplicity, assuming content.text exists.
        if hasattr(content, 'parts'):
            texts = [part.text for part in content.parts if hasattr(part, 'text')]
            return "".join(texts)
        return "" # Or handle other content types as per your original utils.py

    # You might also have public_access_warning, custom_theme, next_steps_html, etc.
    # These are UI-related and won't be part of the core generation logic for pure Python.


def generate_legal_advice(user_message: str, chat_history: list = None, project_id: str = "sodium-coil-470706-f4", location: str = "global"):
    """
    Function to call the model for legal advice based on user input and chat history.

    Args:
        user_message (str): The current message from the user.
        chat_history (list): A list of previous chat messages. Each item in the list
                              should be a dictionary like {"role": "user"|"model", "content": "message text"}.
        project_id (str): Google Cloud project ID.
        location (str): Google Cloud location for Vertex AI.

    Returns:
        str: The generated legal advice from the model.
    """
    if chat_history is None:
        chat_history = []

    # Initialize the genai client
    client = genai.Client(
        vertexai=True,
        project=project_id,
        location=location,
    )

    si_text1 = types.Part.from_text(text="""you are a highly qualified legal professional, renowned for your sharp wit, unparalleled expertise, and ability to win even the toughest cases. As a top-tier legal advisor and document assistant, you are well-versed in all areas of law, including corporate, criminal, civil, tax, intellectual property, international, and regulatory law in the Indian jurisdiction specifically. You provide precise, actionable legal advice, identifying legitimate strategies, exemptions, or loopholes to minimize penalties or liabilities when requested, without ever endorsing illegal actions.""")

    model = "gemini-2.5-flash-lite"

    # Build the conversation history for the model
    contents = []
    for prev_msg in chat_history:
        role = "user" if prev_msg["role"] == "user" else "model"
        parts = utils.get_parts_from_message(prev_msg["content"])
        if parts:
            contents.append(types.Content(role=role, parts=parts))

    # Add the current user message
    if user_message:
        contents.append(types.Content(role="user", parts=utils.get_parts_from_message(user_message)))

    generate_content_config = types.GenerateContentConfig(
        temperature=0.2,
        top_p=0.95,
        max_output_tokens=2000,
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF")
        ],
        system_instruction=[si_text1],
    )

    full_response_text = ""
    # Stream the content from the model
    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        if chunk.candidates and chunk.candidates[0] and chunk.candidates[0].content:
            # We use utils.convert_content_to_gr_type, but adapted to return just text
            # as there's no Gradio UI here.
            text_chunk = utils.convert_content_to_gr_type(chunk.candidates[0].content)
            full_response_text += text_chunk

    return full_response_text

# Example Usage (after you provide utils.py content)
if __name__ == "__main__":
    # This simulates a conversation history
    history = [
        {"role": "user", "content": "What are the common pitfalls in Indian corporate law for startups?"},
        {"role": "model", "content": "Startups in India often face challenges with intellectual property registration, compliance with labor laws, and navigating complex taxation. Ensuring proper legal due diligence from the outset is crucial."}
    ]

    new_query = "Can you elaborate on intellectual property challenges?"

    print("Generating response...")
    response = generate_legal_advice(new_query, chat_history=history)
    print("\nModel Response:")
    print(response)

    print("\n--- New Conversation ---")
    first_query = "What is the process for trademark registration in India?"
    response_new_convo = generate_legal_advice(first_query) # No history for the first turn
    print("\nModel Response:")
    print(response_new_convo)


Generating response...


c:\Users\user\OneDrive\Documents\GenAI-exchange\backend\venv\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)



Model Response:
Ah, intellectual property – the lifeblood of many a startup, and often, its Achilles' heel. You're wise to probe this area. In the Indian context, the pitfalls for startups regarding IP are numerous and can have significant long-term consequences. Let's dissect them:

**1. Neglecting IP Protection Until It's Too Late:**

*   **The "We'll Do It Later" Syndrome:** This is perhaps the most common and damaging pitfall. Startups, in their haste to launch and gain market traction, often postpone formal IP registration. They might believe their idea is too unique to be copied, or that the cost and effort of registration are prohibitive.
*   **Consequences:** By the time they realize the need for protection, their innovations might already be in the public domain, or worse, copied and patented by a competitor. This makes it incredibly difficult, if not impossible, to secure exclusive rights later.

**2. Inadequate Understanding of Different IP Rights:**

*   **Confusing Tradem

In [5]:
import base64
import mimetypes
import io
import typing
from PIL import Image # For handling image data
from google import genai
from google.genai import types

# --- Start of utils.py content, adapted for pure Python ---

def get_part_from_file(file_path):
  """Help function to get the part from a file."""
  guessed_type = mimetypes.guess_type(file_path)
  if guessed_type:
    mime_type = guessed_type[0]
  else:
    mime_type = "application/octet-stream"
  with open(file_path, "rb") as f:
    data = f.read()
    return types.Part.from_bytes(
        data=data,
        mime_type=mime_type,
    )


def get_bytes_from_image(image: Image.Image, mime_type: str = "PNG") -> bytes:
  """Converts a PIL Image object to bytes in the specified format.

  Args:
      image: The PIL Image object.
      mime_type: The image format to save as (e.g., 'PNG', 'JPEG', 'GIF').
        Defaults to 'PNG'.

  Returns:
      A bytes object representing the image in the specified format.
  """
  img_byte_arr = io.BytesIO()
  image.save(img_byte_arr, format=mime_type)
  img_byte_arr = img_byte_arr.getvalue()
  return img_byte_arr


def get_parts_from_message(
    message: typing.Union[str, dict, Image.Image, bytes, typing.Tuple[str, ...]],
):
  """Help function to get the parts from a message.
  Adapted to remove Gradio-specific types.

  Args:
      message: The input message, which can be a string, a dictionary
               (for text and files), a PIL Image object, or bytes (for an image).
               Gradio-specific types like gr.Image are removed.
  """
  parts = []
  if isinstance(message, dict):
    # This assumes a dict could contain {'text': '...', 'files': ['path1', 'path2']}
    if "text" in message and message["text"]:
      parts.append(types.Part.from_text(text=message["text"]))

    if "files" in message:
      for file_path in message["files"]:
        parts.append(get_part_from_file(file_path))

  elif isinstance(message, str):
    if message:
      parts.append(types.Part.from_text(text=message))

  elif isinstance(message, Image.Image): # Direct PIL Image object
    # Default to PNG if format not specified or inferable
    # You might need to pass the actual format if available
    bytes_data = get_bytes_from_image(message, mime_type="PNG")
    parts.append(
        types.Part.from_bytes(data=bytes_data, mime_type="image/png") # Or infer from filename/metadata
    )
  elif isinstance(message, bytes): # Raw image bytes
      # You would need to know the mime_type here, or infer it.
      # For now, making an assumption, you might need to pass this info.
      parts.append(
          types.Part.from_bytes(data=message, mime_type="image/jpeg") # Example, adjust as needed
      )
  elif isinstance(message, tuple): # Assuming a tuple of paths for now, similar to old Gradio behavior
      for item in message:
          if isinstance(item, str): # Could be a file path or text
              # Heuristic: if it looks like a path, treat as file, else text
              if item.startswith('/') or item.startswith('./') or item.startswith('../'): # Simple path check
                  try:
                      parts.append(get_part_from_file(item))
                  except FileNotFoundError:
                      parts.append(types.Part.from_text(text=item)) # Fallback if not a real file
              else:
                  parts.append(types.Part.from_text(text=item))
          else:
              # Handle other types within tuple if necessary, or raise error
              pass


  # To avoid error when sending empty message.
  if not parts:
    parts.append(types.Part.from_text(text=" "))

  return parts


def convert_blob_to_image(blob: types.Blob) -> Image.Image:
  """Converts a blob of image data to a PIL Image object."""
  blob_data = blob.data
  image_stream = io.BytesIO(blob_data)
  image = Image.open(image_stream)
  return image


def image_blob_to_markdown_base64(blob: types.Blob) -> str:
  """Converts image bytes to a Markdown displayable string using Base64 encoding."""
  blob_data = blob.data
  base64_string = base64.b64encode(blob_data).decode("utf-8")
  # Use blob.mime_type directly as provided by the model response
  markdown_string = (
      f'<img src="data:{blob.mime_type};base64,{base64_string}">'
  )
  return markdown_string


def convert_part_to_output_type(
    part: types.Part,
    use_markdown: bool = False,
) -> typing.Optional[typing.Union[str, Image.Image]]:
  """Converts a part object to a str or PIL Image object (no Gradio Image)."""
  if part.text:
    return part.text
  elif part.inline_data:
    if use_markdown:
      return image_blob_to_markdown_base64(part.inline_data)
    # Return a PIL Image object directly if not using markdown
    return convert_blob_to_image(part.inline_data)
  else:
    return None


def convert_content_to_output_list(
    content: typing.Optional[types.Content],
    use_markdown: bool = False,
) -> typing.List[typing.Union[str, Image.Image]]:
  """Converts a content object to a list of strings or PIL Image objects."""
  if content is None or content.parts is None:
    return []

  results = [
      convert_part_to_output_type(part, use_markdown) for part in content.parts
  ]
  return [res for res in results if res is not None]

# --- End of utils.py content, adapted for pure Python ---


# The main generation function, adapted to use the pure Python utils
def generate_legal_advice(
    user_message: typing.Union[str, dict, Image.Image, bytes, typing.Tuple[str, ...]],
    chat_history: typing.Optional[typing.List[typing.Dict[str, typing.Any]]] = None,
    project_id: str = "sodium-coil-470706-f4",
    location: str = "global",
    stream_response: bool = False # Added for potential Flask streaming
):
    """
    Function to call the model for legal advice based on user input and chat history.

    Args:
        user_message: The current message from the user. Can be a string,
                      a dictionary (for text/files), a PIL Image, or raw bytes.
        chat_history: A list of previous chat messages. Each item in the list
                      should be a dictionary like {"role": "user"|"model", "content": "message text"}.
                      The 'content' can also be a more complex type if it was e.g., an image.
        project_id (str): Google Cloud project ID.
        location (str): Google Cloud location for Vertex AI.
        stream_response (bool): If True, yields chunks of the response. If False, returns the full response.

    Returns:
        If stream_response is True, yields string chunks.
        If stream_response is False, returns a single string with the full response.
    """
    if chat_history is None:
        chat_history = []

    # For a Flask app, you might validate keys here, or earlier in middleware.
    # For this pure Python function, we remove the request object dependency.
    # validate_key_result = utils.validate_key(request) # Removed request dependency
    # if validate_key_result is not None:
    #     yield validate_key_result # This would also need to be adapted for non-Gradio streaming.

    client = genai.Client(
        vertexai=True,
        project=project_id,
        location=location,
    )

    si_text1 = types.Part.from_text(text="""you are a highly qualified legal professional, renowned for your sharp wit, unparalleled expertise, and ability to win even the toughest cases. As a top-tier legal advisor and document assistant, you are well-versed in all areas of law, including corporate, criminal, civil, tax, intellectual property, international, and regulatory law in the Indian jurisdiction specifically. You provide precise, actionable legal advice, identifying legitimate strategies, exemptions, or loopholes to minimize penalties or liabilities when requested, without ever endorsing illegal actions.""")

    model = "gemini-2.5-flash-lite"

    contents = []
    # Build the conversation history for the model
    for prev_msg in chat_history:
        role = "user" if prev_msg["role"] == "user" else "model"
        # Use the adapted get_parts_from_message for previous messages' content
        parts = get_parts_from_message(prev_msg["content"])
        if parts:
            contents.append(types.Content(role=role, parts=parts))

    # Add the current user message
    if user_message:
        contents.append(
            types.Content(role="user", parts=get_parts_from_message(user_message))
        )

    generate_content_config = types.GenerateContentConfig(
        temperature=0.2,
        top_p=0.95,
        max_output_tokens=2000,
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF")
        ],
        system_instruction=[si_text1],
    )

    response_generator = client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    )

    if stream_response:
        for chunk in response_generator:
            if chunk.candidates and chunk.candidates[0] and chunk.candidates[0].content:
                # convert_content_to_output_list will give a list, join if it's text
                chunk_parts = convert_content_to_output_list(chunk.candidates[0].content, use_markdown=True)
                # Assuming text for streaming, handle images separately if needed
                text_chunks = [p for p in chunk_parts if isinstance(p, str)]
                if text_chunks:
                    yield "".join(text_chunks)
    else:
        full_response_text = ""
        # If not streaming, collect all parts and return as a single string
        for chunk in response_generator:
            if chunk.candidates and chunk.candidates[0] and chunk.candidates[0].content:
                chunk_parts = convert_content_to_output_list(chunk.candidates[0].content, use_markdown=True)
                # Join text parts; for images, you'd collect them or handle them differently.
                text_content = [p for p in chunk_parts if isinstance(p, str)]
                full_response_text += "".join(text_content)
        return full_response_text

# Example Usage:
if __name__ == "__main__":
    print("--- Testing text-only conversation ---")
    history_text_only = [
        {"role": "user", "content": "What are the key aspects of the Indian Companies Act, 2013 regarding corporate governance?"},
        {"role": "model", "content": "The Indian Companies Act, 2013, emphasizes robust corporate governance through provisions like mandatory independent directors, audit committees, and detailed disclosure requirements to ensure transparency and accountability."}
    ]

    new_query_text = "Can you elaborate on the role of independent directors?"

    print("\nGenerating response (non-streaming):")
    response = generate_legal_advice(new_query_text, chat_history=history_text_only, stream_response=False)
    print(response)

    print("\n--- Testing streaming response ---")
    print("Generating response (streaming):")
    # You might reset history for a fresh streamed conversation
    streaming_history = [
        {"role": "user", "content": "Tell me about intellectual property rights in India, focusing on patents."},
    ]
    streamed_response = generate_legal_advice("What's the typical duration of a patent?", chat_history=streaming_history, stream_response=True)
    full_streamed_output = ""
    for chunk in streamed_response:
        print(chunk, end="") # print each chunk as it arrives
        full_streamed_output += chunk
    print("\n(Full streamed output gathered):", full_streamed_output)

    print("\n--- Testing with a dummy image file (requires an actual file at path) ---")
    # To test this, you would need to create a dummy image file, e.g., "dummy_image.png"
    # For example:
    # from PIL import Image
    # img = Image.new('RGB', (60, 30), color = 'red')
    # img.save("dummy_image.png")
    try:
        # Create a dummy image file for testing
        dummy_img = Image.new('RGB', (60, 30), color = 'red')
        dummy_img.save("dummy_image.png")

        image_query = {"text": "Describe this image in the context of legal documents.", "files": ["./dummy_image.png"]}
        print("\nGenerating response with image (non-streaming):")
        # Note: Gemini-Flash models are better at text, for strong vision use Gemini-Pro-Vision.
        # This example is to show the structure, not necessarily the output quality for a small dummy image.
        image_response = generate_legal_advice(image_query, stream_response=False)
        print(image_response)

    except FileNotFoundError:
        print("\nSkipping image test: 'dummy_image.png' not found. Please create one to test image input.")
    finally:
        # Clean up the dummy image file
        import os
        if os.path.exists("dummy_image.png"):
            os.remove("dummy_image.png")


--- Testing text-only conversation ---

Generating response (non-streaming):
<generator object generate_legal_advice at 0x00000234F6F2CC10>

--- Testing streaming response ---
Generating response (streaming):
Ah, patents! A fascinating and often fiercely contested arena within the realm of intellectual property. In India, the legal framework governing patents is primarily laid out in the **Patents Act, 1970**, and the rules framed thereunder. It's a robust system designed to encourage innovation by granting exclusive rights to inventors for their novel and inventive creations.

Let's delve into the specifics of patents in India, and then I'll address your question about their duration.

### Understanding Patents in India

A patent is essentially a statutory right granted by the government to an inventor, giving them the exclusive right to prevent others from making, using, selling, or importing their invention for a limited period. To be patentable in India, an invention must generally

In [ ]:
# Example Usage:
if __name__ == "__main__":
    print("--- Testing text-only conversation ---")
    history_text_only = [
        {"role": "user", "content": "What are the key aspects of the Indian Companies Act, 2013 regarding corporate governance?"},
        {"role": "model", "content": "The Indian Companies Act, 2013, emphasizes robust corporate governance through provisions like mandatory independent directors, audit committees, and detailed disclosure requirements to ensure transparency and accountability."}
    ]

    new_query_text = "Can you elaborate on the role of independent directors?"

    print("\nGenerating response (non-streaming):")
    response = generate_legal_advice(new_query_text, chat_history=history_text_only, stream_response=False)
    print(response)

    print("\n--- Testing streaming response ---")
    print("Generating response (streaming):")
    # You might reset history for a fresh streamed conversation
    streaming_history = [
        {"role": "user", "content": "Tell me about intellectual property rights in India, focusing on patents."},
    ]
    streamed_response = generate_legal_advice("What's the typical duration of a patent?", chat_history=streaming_history, stream_response=True)
    full_streamed_output = ""
    for chunk in streamed_response:
        print(chunk, end="") # print each chunk as it arrives
        full_streamed_output += chunk
    print("\n(Full streamed output gathered):", full_streamed_output)

    print("\n--- Testing with a dummy image file (requires an actual file at path) ---")
    # To test this, you would need to create a dummy image file, e.g., "dummy_image.png"
    # For example:
    # from PIL import Image
    # img = Image.new('RGB', (60, 30), color = 'red')
    # img.save("dummy_image.png")
    try:
        # Create a dummy image file for testing
        dummy_img = Image.new('RGB', (60, 30), color = 'red')
        dummy_img.save("dummy_image.png")

        image_query = {"text": "Describe this image in the context of legal documents.", "files": ["./dummy_image.png"]}
        print("\nGenerating response with image (non-streaming):")
        # Note: Gemini-Flash models are better at text, for strong vision use Gemini-Pro-Vision.
        # This example is to show the structure, not necessarily the output quality for a small dummy image.
        image_response = generate_legal_advice(image_query, stream_response=False)
        print(image_response)

    except FileNotFoundError:
        print("\nSkipping image test: 'dummy_image.png' not found. Please create one to test image input.")
    finally:
        # Clean up the dummy image file
        import os
        if os.path.exists("dummy_image.png"):
            os.remove("dummy_image.png")

In [6]:
{
  "cell_type": "code",
  "metadata": {
    "language": "python"
  },
  "source": [
    "import sys",
    "from PIL import Image",
    "",
    "def automated_chat(stream_response=False, image_mode=False):",
    "    history = []",
    "    print(\"\\n--- Automated Legal Chat ---\")",
    "    print(\"Type 'exit' to end the chat.\")",
    "    while True:",
    "        if image_mode:",
    "            img_path = input(\"Enter image file path (or leave blank for text only): \").strip()",
    "            if img_path:",
    "                try:",
    "                    img = Image.open(img_path)",
    "                    user_input = {\"text\": input(\"Your question: \"), \"files\": [img_path]}",
    "                except Exception as e:",
    "                    print(f\"Error loading image: {e}\")",
    "                    continue",
    "            else:",
    "                user_input = input(\"Your question: \")",
    "        else:",
    "            user_input = input(\"Your question: \")",
    "",
    "        if isinstance(user_input, str) and user_input.lower() == \"exit\":",
    "            print(\"Exiting chat.\")",
    "            break",
    "",
    "        history.append({\"role\": \"user\", \"content\": user_input})",
    "",
    "        print(\"Model response:\")",
    "        if stream_response:",
    "            response_stream = generate_legal_advice(user_input, chat_history=history, stream_response=True)",
    "            full_response = \"\"",
    "            for chunk in response_stream:",
    "                print(chunk, end=\"\")",
    "                full_response += chunk",
    "            print()",
    "            history.append({\"role\": \"model\", \"content\": full_response})",
    "        else:",
    "            response = generate_legal_advice(user_input, chat_history=history, stream_response=False)",
    "            print(response)",
    "            history.append({\"role\": \"model\", \"content\": response})",
    "",
    "    print(\"\\n--- Chat session ended ---\")"
  ]
}

{'cell_type': 'code',
 'metadata': {'language': 'python'},
 'source': ['import sys',
  'from PIL import Image',
  '',
  'def automated_chat(stream_response=False, image_mode=False):',
  '    history = []',
  '    print("\\n--- Automated Legal Chat ---")',
  '    print("Type \'exit\' to end the chat.")',
  '    while True:',
  '        if image_mode:',
  '            img_path = input("Enter image file path (or leave blank for text only): ").strip()',
  '            if img_path:',
  '                try:',
  '                    img = Image.open(img_path)',
  '                    user_input = {"text": input("Your question: "), "files": [img_path]}',
  '                except Exception as e:',
  '                    print(f"Error loading image: {e}")',
  '                    continue',
  '            else:',
  '                user_input = input("Your question: ")',
  '        else:',
  '            user_input = input("Your question: ")',
  '',
  '        if isinstance(user_input, str) an